<a href="https://colab.research.google.com/github/AryaJeet1364/PyTorch_Projects/blob/main/LinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Manual vs Built-in

In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# To make sure results are the same every time
torch.manual_seed(42)
np.random.seed(42)

In [3]:
# Loading California housing data
data = fetch_california_housing()
X, y = data.data, data.target

In [4]:
# Splitting into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Scale features so training goes smoother
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
# Convert to torch tensors
X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train).unsqueeze(1)  # to make it (n, 1)
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test).unsqueeze(1)

Manual

In [7]:
input_dim = X_train.shape[1]  # Number of features
W = torch.randn(input_dim, 1, dtype=torch.float32) * 0.01  # Small random weights init
b = torch.zeros(1, dtype=torch.float32)  # Bias initialized to zero

lr = 0.01  # Learning rate
epochs = 1000  # Training iterations
n = X_train.shape[0]  # Number of samples

for epoch in range(epochs):
    y_pred = X_train.mm(W) + b  # Predict output
    loss = ((y_pred - y_train) ** 2).mean()  # Compute MSE loss

    # Calculating gradients manually
    grad_y_pred = 2.0 * (y_pred - y_train) / n
    grad_W = X_train.t().mm(grad_y_pred)  # Gradient for weights
    grad_b = grad_y_pred.sum()  # Gradient for bias

    # Update parameters using gradient descent
    W -= lr * grad_W
    b -= lr * grad_b

    # Printing loss every 100 epochs
    if epoch % 100 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Training MSE: {loss.item():.4f}")

final_train_loss = loss.item()  # Final training loss

# Evaluate test loss without tracking gradients
with torch.no_grad():
    y_test_pred = X_test.mm(W) + b
    test_loss = ((y_test_pred - y_test) ** 2).mean().item()

print("\n== Manual Linear Regression ==")
print(f"Final Training MSE: {final_train_loss:.4f}")
print(f"Test MSE: {test_loss:.4f}")


Epoch 1/1000, Training MSE: 5.6304
Epoch 101/1000, Training MSE: 0.7116
Epoch 201/1000, Training MSE: 0.5962
Epoch 301/1000, Training MSE: 0.5739
Epoch 401/1000, Training MSE: 0.5589
Epoch 501/1000, Training MSE: 0.5481
Epoch 601/1000, Training MSE: 0.5401
Epoch 701/1000, Training MSE: 0.5343
Epoch 801/1000, Training MSE: 0.5301
Epoch 901/1000, Training MSE: 0.5270

== Manual Linear Regression ==
Final Training MSE: 0.5247
Test MSE: 0.5547


Using NN module

In [8]:
# -------- PyTorch nn.Linear Model --------
class TorchLinear(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        return self.linear(x)

model = TorchLinear(X_train.shape[1])
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Train nn.Linear model
for i in range(1000):
    preds = model(X_train)
    loss = loss_fn(preds, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if(i%100 == 0):
        print(f"Epoch {i+1}/{1000}, Training MSE: {loss.item():.4f}")

final_train_loss_nn = loss.item()

# Test loss
with torch.no_grad():
    preds_test_nn = model(X_test)
    test_loss_nn = loss_fn(preds_test_nn, y_test).item()

print("== PyTorch nn.Linear ==")
print(f"Final Training MSE: {final_train_loss_nn:.4f}")
print(f"Test MSE: {test_loss_nn:.4f}")

Epoch 1/1000, Training MSE: 6.6704
Epoch 101/1000, Training MSE: 0.7193
Epoch 201/1000, Training MSE: 0.5898
Epoch 301/1000, Training MSE: 0.5699
Epoch 401/1000, Training MSE: 0.5568
Epoch 501/1000, Training MSE: 0.5472
Epoch 601/1000, Training MSE: 0.5400
Epoch 701/1000, Training MSE: 0.5347
Epoch 801/1000, Training MSE: 0.5307
Epoch 901/1000, Training MSE: 0.5278
== PyTorch nn.Linear ==
Final Training MSE: 0.5255
Test MSE: 0.5588
